In [ ]:
import os
import sys
import matplotlib.pyplot as plt
import numpy as np

"""
このnotebookがbaseフォルダを読めるようになるための処理
"""

root_rel = '../../'

# 相対パスを絶対パスに変換してsys.pathに追加
root_abs = os.path.abspath(root_rel)
if root_abs not in sys.path:
    sys.path.append(root_abs)
print(root_abs)


In [ ]:
import h5py

import time
from tqdm import tqdm

# テストデータのpath

In [ ]:
nxs_path = "OIbDia08_6_00000.nxs"
poni_path = "pyFAI_calib2_MgS400_CeO2_240511.poni"

# nxsクラス

In [ ]:
# nxsインスタンス
from model.XRD.Nxs import NxsFile
nxs = NxsFile(filepath=nxs_path)

In [ ]:
nxs.set_poni(poni_path=poni_path)

In [ ]:
nxs.read_file()

In [ ]:
def show_hdf5_hierarchy(f, indent=0, path=""):
    for key in f.keys():
        print(" " * indent + "/" + key)  # グループ名を出力
        current_path = path + "/" + key
        if isinstance(f[key], h5py.Group): # もしkey先がgroupの場合
            show_hdf5_hierarchy(f[key], indent + 4, path=current_path)  # グループ内のさらに深い階層を出力
        else:
            print(" " * indent + f" --> path:  \"{current_path}\"")
            print(" " * indent + f"       L {type(f[key])}")
            dataset = f[key]

            if dataset.shape == (): # スカラー(単一値)の場合
                value = str(dataset[()]) # スカラーの場合の読み取り
            else:
                value = str(f.get(key)[0])

            if len(value) > 20:
                print(" " * indent + f"       L {value[:20]}")
            else:
                print(" " * indent + f"       L {value}")

In [ ]:
show_hdf5_hierarchy(nxs.data_file)

# 取り出すものメモ

とりあえず意味ありそうなものコピペして、優先順位低い(今は使いそうにない)ものにはxを、使うものにはoをつける

1 x
            /acquisition_mode
             --> path:  "/entry/instrument/detector/acquisition_mode"
                   L <class 'h5py._hl.dataset.Dataset'>
                   L b'ContinuousReadWrit

2 x
            /bit_depth_readout
             --> path:  "/entry/instrument/detector/bit_depth_readout"
                   L <class 'h5py._hl.dataset.Dataset'>
                   L 12

3 o

                /frame_numbers
                 --> path:  "/entry/instrument/detector/collection/frame_numbers"
                       L <class 'h5py._hl.dataset.Dataset'>
                       L 4000
                       
4 o

                /save_file_name
                 --> path:  "/entry/instrument/detector/collection/save_file_name"
                       L <class 'h5py._hl.dataset.Dataset'>
                       L b'OIbDia08_6_00000'
                       
5 o

            /count_time
             --> path:  "/entry/instrument/detector/count_time"
                   L <class 'h5py._hl.dataset.Dataset'>
                   L 5.0
                       
6 x
                /threshold_energy
                 --> path:  "/entry/instrument/detector/collection/threshold_energy"
                       L <class 'h5py._hl.dataset.Dataset'>
                       L 15000.0
                       
7 o

            /data
             --> path:  "/entry/instrument/detector/data"
                   L <class 'h5py._hl.dataset.Dataset'>
                   L [[0 0 0 ... 1 3 4]
                   
8 x
            /description
             --> path:  "/entry/instrument/detector/description"
                   L <class 'h5py._hl.dataset.Dataset'>
                   L b'Lambda'
                   
9 x
            /detector_readout_time
             --> path:  "/entry/instrument/detector/detector_readout_time"
                   L <class 'h5py._hl.dataset.Dataset'>
                   L 0.0
                   
10 x
            /pixelmask
             --> path:  "/entry/instrument/detector/pixelmask"
                   L <class 'h5py._hl.dataset.Dataset'>
                   L [[0 0 0 ... 0 0 0]
                   
11 o

            /saturation_value
             --> path:  "/entry/instrument/detector/saturation_value"
                   L <class 'h5py._hl.dataset.Dataset'>
                   L 4095
                   
12 x
            /sensor_material
             --> path:  "/entry/instrument/detector/sensor_material"
                   L <class 'h5py._hl.dataset.Dataset'>
                   L b'CdTe'
                   
13 x
            /sensor_thickness
             --> path:  "/entry/instrument/detector/sensor_thickness"
                   L <class 'h5py._hl.dataset.Dataset'>
                   L 300.0
                   
14 x
            /type
             --> path:  "/entry/instrument/detector/type"
                   L <class 'h5py._hl.dataset.Dataset'>
                   L b'photon counting'
                   
15 o

            /x_pixel_size
             --> path:  "/entry/instrument/detector/x_pixel_size"
                   L <class 'h5py._hl.dataset.Dataset'>
                   L 55.0
                   
16 o

            /y_pixel_size
             --> path:  "/entry/instrument/detector/y_pixel_size"
                   L <class 'h5py._hl.dataset.Dataset'>
                   L 55.0

# データを取り出す準備

In [ ]:
def return_data(f: h5py.File, data_path: str, shape: list = None): # あとでクラスに実装
    dataset = f[data_path]

    if dataset.shape == (): # スカラー(単一値)の場合
        print("This is scalar.\n")
        value = dataset[()] # スカラーの場合の読み取り
    else:
        if shape is None: # スライス指定がない場合
            value = f.get(data_path)[:]
        else: # スライス指定がある場合
            value = f.get(data_path)[tuple(shape)] # 部分的に返す

    return value

In [ ]:
# データpathコピペがめんどくさいので、クエリで引っかかるものがあればそのpathをリストで返す関数を作る

from model.File.HDF5 import HDF5
hdf = HDF5(hdf = nxs.data_file)

In [ ]:
# 部分的な指定の場合
# cakingとintegrationはこれを使う
data_path = hdf.search_data_path(query = 'data')[0]
partial_img = return_data(nxs.data_file, data_path=data_path, shape=[0])
partial_img.shape

# 取り出していく

In [ ]:
print(hdf.search_data_path(query = 'save_file_name'))

In [ ]:
# ファイル名
data_path = hdf.search_data_path(query = 'save_file_name')[0]
filename = return_data(nxs.data_file, data_path=data_path)

filename = filename[0]

print(f"{filename = }")
print(f"{type(filename) = }\n")

filename = filename.decode("utf-8")

print(f"{filename = }")
print(f"{type(filename) = }")

In [ ]:
# frame_num
data_path = hdf.search_data_path(query = 'frame_num')[0]

frame_num = return_data(nxs.data_file, data_path=data_path)

frame_num = frame_num[0]

print(f"{frame_num = }")
print(f"{type(frame_num) = }\n")


In [ ]:
# count_time
data_path = hdf.search_data_path(query = 'count_time')[0]

count_time = return_data(nxs.data_file, data_path=data_path)

count_time = count_time[0]

count_time, type(count_time)

In [ ]:
# img
data_path = hdf.search_data_path(query = 'data')[0]

img = return_data(nxs.data_file, data_path=data_path)

# print("stop")
# time.sleep(10)
# print("resume")

# 消しちゃう(こともあった)ので、ここで定義しておく
x_shape = img.shape[1]
y_shape = img.shape[2]

# del img

In [ ]:
# saturation_value
data_path = hdf.search_data_path(query = 'saturation_value')[0]

saturation_value = return_data(nxs.data_file, data_path=data_path)[0] # shapeが(1,)なら、shapeオプションでなくこのように指定してOK

print(f"{saturation_value = }")

In [ ]:
# x_pixel_size
data_path = hdf.search_data_path(query = 'x_pixel_size')[0]

x_pixel_size = return_data(nxs.data_file, data_path=data_path)[0]

print(f"{x_pixel_size = }")

In [ ]:
# y_pixel_size

data_path = hdf.search_data_path(query = 'y_pixel_size')[0]

y_pixel_size = return_data(nxs.data_file, data_path=data_path)[0]

print(f"{y_pixel_size = }")



# 格納データを作成する

In [ ]:
# exposure_ms

exposure_ms = count_time / 1_000

print(f"{exposure_ms = }")

In [ ]:
# fps
fps = 1 / exposure_ms

print(f"{fps = }")

In [ ]:
# x_shape
# 定義はimgでやってる

print(f"{x_shape = }")

In [ ]:
# y_shape
# 定義はimgでやってる

print(f"{y_shape = }")

In [ ]:
# cakingとintegration
# メモリ見ながら試す。爆発しそうなら何とか分ける
# 温度とかも格納することを考えると、保存し終わった後にオブジェクトを消して、ちゃんとメモリが解放されいてることを確かめる必要がある
npt_rad = 2000
npt_azi = 360

cake = np.zeros((frame_num, npt_azi, npt_rad))
I = np.zeros((frame_num, npt_rad))

data_path = hdf.search_data_path(query = 'data')[0]

for frame in tqdm(range(frame_num)):
    # 1frame分のデータを取得
    partial_img = return_data(nxs.data_file, data_path=data_path, shape=[frame])

    # caking
    cake_tmp, tth, azi = nxs.ai.integrate2d(partial_img,
                                            npt_rad=npt_rad, # NOTE これはintegrate_1dと揃える
                                            npt_azim=npt_azi,
                                            unit="2th_deg") # ここでmask渡す
    cake[frame] = cake_tmp
    
    # integration
    tth, I_tmp = nxs.ai.integrate1d(partial_img,
                                    npt=npt_rad,
                                    unit="2th_deg") # ここでmask渡す
    I[frame] = I_tmp


# cakingとintegrationができているかplotで確認

In [ ]:
import matplotlib.pyplot as plt
plt.imshow(I, aspect='auto')

In [ ]:
plt.imshow(cake[0], aspect='auto')

In [ ]:
plt.plot(I[0])

# 格納する

In [ ]:
# テスト用のHDFファイルを作る
# WARNING 初期化も兼ねてるので注意

hdf5_name = 'hdf/test_nxs.hdf5'

base_layer = 'entry/'

test_layer = base_layer + 'test_str'

with h5py.File(hdf5_name, "w") as f:
    f.create_dataset(test_layer, data="nxs_test")

In [ ]:
with h5py.File(hdf5_name, "r") as f:
    show_hdf5_hierarchy(f=f)

In [ ]:
# layerを作る
layer_var = [ # これを変数名に指定れば、execで一括実行できる
    'nxs_path',
    'poni_path',
    'filename',
    'frame_num',
    'count_time',
    'img',
    'saturation_value',
    'x_pixel_size',
    'y_pixel_size',
    'exposure_ms',
    'fps',
    'x_shape',
    'y_shape',
    'cake',
    'I'
]

layer_list = []

for layer in layer_var:
    layer_name = base_layer + 'nxs/' + layer
    layer_list.append(layer_name)

layer_list

In [ ]:
layer_list[0]

In [ ]:
# 書き込み
compression_list = ['img', 'cake', 'I']

with h5py.File(hdf5_name, "a") as f:
    for i in range(len(layer_var)):
        if layer_var[i] in compression_list: # 圧縮する場合
            # NOTE とりあえずお任せにしている部分が多い。さらに圧縮したり適切なchunk sizeを決められたりする(読み出しに時間がかかるようになる)
            exec(f"f.create_dataset(\"{layer_list[i]}\", data={layer_var[i]}, compression='gzip')")
        else:
            exec(f"f.create_dataset(\"{layer_list[i]}\", data={layer_var[i]})")


In [ ]:
with h5py.File(hdf5_name, "r") as f:
    show_hdf5_hierarchy(f=f)

# テストとして読み出し

In [ ]:
with h5py.File(hdf5_name, "r") as f:
    arr = return_data(f=f, data_path='entry/nxs/cake', shape=[0])
arr.shape

In [ ]:
plt.imshow(arr, aspect='auto')

# でかい変数が残ってれば消す

In [ ]:
# この検索は完全じゃなさそう
# 部分被りでも分岐に入ってしまいそう
# TODO 改善

if 'cake' in locals():
    del cake
if 'I' in locals():
    del I
if 'img' in locals():
    del img

# スプライン補間を試す

In [ ]:
hdf5_name = 'hdf/test_nxs.hdf5'

with h5py.File(hdf5_name, "r") as f:
    arr = return_data(f=f, data_path='entry/nxs/I', shape=[0])
arr.shape


In [ ]:
import matplotlib.pyplot as plt
plt.plot(arr)

In [ ]:
from scipy.signal import savgol_filter

In [ ]:
background = savgol_filter(arr, 101, 2) # y, window, order

In [ ]:
plt.plot(arr)
plt.plot(background)

In [ ]:
from xypattern.pattern import SmoothBrucknerBackground
from xypattern import Pattern

In [ ]:
x = np.arange(arr.shape[0])
bkg = [arr]

auto_bkg = SmoothBrucknerBackground(*(0.1, 150, 50))

for i in range(3):
    bkg.append(auto_bkg.extract_background(Pattern(x, bkg[i])))

In [ ]:
plt.plot(arr)
for i in range(len(bkg)):
    plt.plot(bkg[i], label=f"{i}")
plt.legend()


In [ ]:
plt.plot(bkg[3] - bkg[2])

## ↓これがdioptasと同じっぽい？よくfittingできてる

In [ ]:
# https://github.com/SHDShim/PeakPo/blob/4c522e147e7715bceba218de58ee185cccd2055e/peakpo/ds_powdiff/background.py
import numpy as np


def fit_bg_cheb_auto(x, y_obs, n_points=20, n_iteration=10, n_cheborder=20,
                     accurate=True):
    """
    this returns cheb parameter for fitted background
    best for synchrotron XRD is:
        N_points = 20, N_iteration = 10, N_cheborder = 20
    :param x: x
    :param y_obs: observed y
    :param n_points:
    :param n_iteration:
    :param n_cheborder:
    :param accurate:
    """
    y_bg_smooth = smooth_bruckner(x, y_obs, n_points, n_iteration)

    # get cheb input parameters
    x_cheb = 2. * (x - x[0]) / (x[-1] - x[0]) - 1.
    cheb_parameters = np.polynomial.chebyshev.chebfit(
        x_cheb, y_bg_smooth, n_cheborder)
    if accurate:
        return np.polynomial.chebyshev.chebval(x_cheb, cheb_parameters)
    else:
        return cheb_parameters


def smooth_bruckner(x, y_obs, n_smooth, n_iter):
    y_original = y_obs

    n_data = y_obs.size
    n = n_smooth
    y = np.empty(n_data + n + n)

    y[n:n + n_data] = y_original[0:n_data]
    y[0:n].fill(y_original[n])
    y[n + n_data:n_data + n + n].fill(y_original[-1])
    y_new = y

    y_avg = np.average(y)
    y_min = np.min(y)

    y_c = y_avg + 2. * (y_avg - y_min)

    y[np.where(y > y_c)] = y_c

    for j in range(0, n_iter):
        for i in range(n, n_data - 1 - n - 1):
            y_new[i] = np.min([y[i], np.average(y[i - n:i + n + 1])])
        y = y_new

    return y[n:n + n_data]

In [ ]:
bkg2 = fit_bg_cheb_auto(x=x, y_obs=arr)

In [ ]:
plt.plot(arr)
plt.plot(bkg2)

In [ ]:
plt.plot(arr-bkg2)

# Mask処理を試す

In [ ]:
mask_path = '/Users/ishizawaosamu/work/ipynb/BL10XU_notebook/base/data/lambda_mask.npy' # pyFAI-calib2で、強度10未満をマスク
mask = np.load(mask_path)

In [ ]:
mask.shape

In [ ]:
hdf5_name = 'hdf/test_nxs.hdf5'

with h5py.File(hdf5_name, "r") as f:
    data = return_data(f=f, data_path='entry/nxs/img', shape=[0])
    poni_path = return_data(f=f, data_path='entry/nxs/poni_path').decode('utf-8')
data.shape, poni_path


In [ ]:
plt.imshow(data)

In [ ]:
from pyFAI.azimuthalIntegrator import AzimuthalIntegrator
ai = AzimuthalIntegrator().load(poni_path)

In [ ]:
tth, I_tmp = ai.integrate1d(data, npt=1000, unit='2th_deg', mask=mask)

In [ ]:
I_tmp.shape

In [ ]:
plt.plot(I_tmp)

In [ ]:
# background引いてみる
x = np.arange(I_tmp.shape[0])
bkg2 = fit_bg_cheb_auto(x=x, y_obs=I_tmp)

plt.plot(I_tmp)
plt.plot(bkg2)

In [ ]:
# cakingもしてみる
import pandas as pd
cake_tmp, tth, azi = ai.integrate2d(data, npt_rad=1000, npt_azim=360, unit='2th_deg', mask=mask)
df = pd.DataFrame(cake_tmp)

In [ ]:
plt.imshow(cake_tmp, aspect='auto') # maskされてない？

In [ ]:
plt.plot(df.sum(axis=0))